# Creating a 1000G reference panel

This creates the 1000G reference panel to merge with the other genetics data to determine the ancestry of the samples.

From the 1000g reference from https://www.cog-genomics.org/plink/2.0/resources#1kg_phase3, only keep the biallelic snps on autosomes with a MAF > 0.01, geno > 0.95 and hwe > 1e-6. Also, pallindromes and long LD regions were excluded. 



In [6]:
%%bash
# decompress pgen and rename psam as suggested
plink2 --zst-decompress all_hg38.pgen.zst > all_hg38.pgen
cp hg38_corrected.psam all_hg38.psam

In [ ]:
%%bash
# sample info
wget http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/working/20130606_sample_info/20130606_sample_info.xlsx

In [3]:
import pandas as pd
t=pd.read_excel('20130606_sample_info.xlsx', usecols=['Sample', 'Population'])
t.rename(columns={'Sample':'IID'}, inplace=True)
t.to_csv('/data/iwakih2/resources/1kg_p3/20130606_sample_info.txt', index=False, sep='\t')

/usr/local/Anaconda/envs/py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [24]:
%%bash
# hg38 long LD region
wget https://raw.githubusercontent.com/meyer-lab-cshl/plinkQC/master/inst/extdata/high-LD-regions-hg38-GRCh38.txt
sed -i 's/chr//g' high-LD-regions-hg38-GRCh38.txt

# hg38 fasta file
wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

--2023-11-06 17:29:30--  https://raw.githubusercontent.com/meyer-lab-cshl/plinkQC/master/inst/extdata/high-LD-regions-hg38-GRCh38.txt
Resolving dtn05-e0 (dtn05-e0)... 10.1.200.241
Connecting to dtn05-e0 (dtn05-e0)|10.1.200.241|:3128... connected.


Proxy request sent, awaiting response... 200 OK
Length: 1137 (1.1K) [text/plain]
Saving to: ‘high-LD-regions-hg38-GRCh38.txt’

     0K .                                                     100% 5.24M=0s

2023-11-06 17:29:30 (5.24 MB/s) - ‘high-LD-regions-hg38-GRCh38.txt’ saved [1137/1137]



In [49]:
%%bash
# filter the snps
plink2\
  --autosome\
  --allow-extra-chr\
  --exclude bed0 high-LD-regions-hg38-GRCh38.txt\
  --fa /data/iwakih2/resources/hg38.fa.gz\
  --geno 0.05\
  --hwe 0.000001\
  --maf 0.01\
  --make-pgen\
  --max-alleles 2\
  --mind 0.01\
  --out all_hg38_filtered\
  --pfile all_hg38 vzs\
  --ref-from-fa force\
  --remove deg2_hg38.king.cutoff.out.id\
  --snps-only just-acgt\
  --sort-vars

PLINK v2.00a3.6LM 64-bit Intel (14 Aug 2022)   www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to all_hg38_filtered.log.
Options in effect:
  --allow-extra-chr
  --autosome
  --exclude bed0 high-LD-regions-hg38-GRCh38.txt
  --fa /data/iwakih2/resources/hg38.fa.gz
  --geno 0.05
  --hwe 0.000001
  --maf 0.01
  --make-pgen
  --max-alleles 2
  --mind 0.01
  --out all_hg38_filtered
  --pfile all_hg38 vzs
  --ref-from-fa force
  --remove deg2_hg38.king.cutoff.out.id
  --snps-only just-acgt
  --sort-vars

Start time: Mon Nov  6 18:07:24 2023
515436 MiB RAM detected; reserving 257718 MiB for main workspace.
Using up to 128 threads (change this with --threads).
3202 samples (1603 females, 1599 males; 2583 founders) loaded from
all_hg38.psam.
61599150 out of 75193455 variants loaded from all_hg38.pvar.zst.
2 categorical phenotypes loaded.
--exclude bed0: 2079938 variants excluded.
--remove: 2573 samples remaining.
Calculating

In [50]:
!grep '##' all_hg38_filtered.pvar | wc -l

211


In [51]:
df = pd.read_csv('all_hg38_filtered.pvar', delim_whitespace=True, skiprows=211, engine='c')
df.head()

,#CHROM,POS,ID,REF,ALT,INFO
0,1,10583,1:10583:G:A,G,A,AC=193;AF=0.0301374;CM=0.00022389;AN=6404;AN_E...
1,1,13752,1:13752:T:C,T,C,AC=202;AF=0.0315428;CM=0.0039001;AN=6404;AN_EA...
2,1,13757,1:13757:G:A,G,A,AC=111;AF=0.0173329;CM=0.0039059;AN=6404;AN_EA...
3,1,14429,1:14429:G:C,G,C,AC=67;AF=0.0104622;CM=0.00468545;AN=6404;AN_EA...
4,1,14436,1:14436:G:A,G,A,AC=96;AF=0.0149906;CM=0.00469357;AN=6404;AN_EA...


In [52]:
df=df[['#CHROM','POS', 'ID', 'REF', 'ALT']].copy()
df = df.replace(['A', 'C', 'G', 'T'], [-10, -1, 1, 10]) # convert to values
IDexclude = df.loc[df['ALT'] + df['REF'] ==0, 'ID'] # 0 = A/T or C/G
IDexclude.to_csv('palindrome.txt', index=False, header=False)
print(IDexclude.shape)

(1512321,)


In [53]:
%%bash
# rename the snps
plink2\
  --pfile all_hg38_filtered\
  --exclude palindrome.txt\
  --out all_hg38_filtered_chrpos\
  --set-all-var-ids 'chr@:#:$r:$a'\
  --make-bed

PLINK v2.00a3.6LM 64-bit Intel (14 Aug 2022)   www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to all_hg38_filtered_chrpos.log.
Options in effect:
  --exclude palindrome.txt
  --make-bed
  --out all_hg38_filtered_chrpos
  --pfile all_hg38_filtered
  --set-all-var-ids chr@:#:$r:$a

Start time: Mon Nov  6 18:18:53 2023
515436 MiB RAM detected; reserving 257718 MiB for main workspace.
Using up to 128 threads (change this with --threads).
2573 samples (1296 females, 1277 males; 2561 founders) loaded from
all_hg38_filtered.psam.
9494396 variants loaded from all_hg38_filtered.pvar.
2 categorical phenotypes loaded.
--exclude: 9494396 variants remaining.
9494396 variants remaining after main filters.
Writing all_hg38_filtered_chrpos.fam ... done.
Writing all_hg38_filtered_chrpos.bim ... done.
Writing all_hg38_filtered_chrpos.bed ... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455

In [54]:
!du -sh *

128K	20130606_sample_info.txt
1.0M	20130606_sample_info.xlsx
5.7G	all_hg38_filtered_chrpos.bed
329M	all_hg38_filtered_chrpos.bim
128K	all_hg38_filtered_chrpos.fam
512	all_hg38_filtered_chrpos.log
512	all_hg38_filtered.log
2.8G	all_hg38_filtered.pgen
128K	all_hg38_filtered.psam
11G	all_hg38_filtered.pvar
8.9G	all_hg38.pgen
3.2G	all_hg38.pgen.zst
128K	all_hg38.psam
2.6G	all_hg38.pvar.zst
128K	deg2_hg38.king.cutoff.out.id
128K	hg38_corrected.psam
939M	hg38.fa.gz
512	high-LD-regions-hg38-GRCh38.txt
23M	palindrome.txt
128K	readme.ipynb
